In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


In [ ]:
import tensorflow_datasets as tfds

In [ ]:
(train_data, test_data), ds_info = tfds.load('tf_flowers',
                                             split=['train[:80%]', 'train[80%:]'],
                                             shuffle_files=True,
                                             as_supervised=True,
                                             with_info=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, label

In [ ]:
train_data = train_data.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(ds_info.features['label'].num_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=30)

Epoch 1/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 37s 260ms/step - accuracy: 0.2896 - loss: 1.7558 - val_accuracy: 0.3896 - val_loss: 1.5209
Epoch 2/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 22s 122ms/step - accuracy: 0.3710 - loss: 1.4821 - val_accuracy: 0.3965 - val_loss: 1.4842
Epoch 3/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 111ms/step - accuracy: 0.4031 - loss: 1.4343 - val_accuracy: 0.4319 - val_loss: 1.4109
Epoch 4/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 12s 127ms/step - accuracy: 0.4290 - loss: 1.3958 - val_accuracy: 0.4373 - val_loss: 1.3705
Epoch 5/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 19s 110ms/step - accuracy: 0.4422 - loss: 1.3606 - val_accuracy: 0.4292 - val_loss: 1.3612
Epoch 6/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 114ms/step - accuracy: 0.4527 - loss: 1.3337 - val_accuracy: 0.4046 - val_loss: 1.3653
Epoch 7/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 118ms/step - accuracy: 0.4602 - loss: 1.3118 - val_accuracy: 0.4005 - val_loss: 1.3727
Epoch 8/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 20s 114ms/step - accuracy: 0.4684 - loss: 1.2918 - val_accu

In [ ]:
base_model.trainable = True

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
fine_tune_history = model.fit(train_data,
                              validation_data=test_data,
                              epochs=15)

Epoch 1/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 124s 702ms/step - accuracy: 0.3444 - loss: 24.9811 - val_accuracy: 0.1635 - val_loss: 15.8607
Epoch 2/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 30s 324ms/step - accuracy: 0.8188 - loss: 2.3234 - val_accuracy: 0.1635 - val_loss: 26.8555
Epoch 3/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 323ms/step - accuracy: 0.9330 - loss: 0.5944 - val_accuracy: 0.2166 - val_loss: 24.5463
Epoch 4/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 40s 315ms/step - accuracy: 0.9330 - loss: 0.4408 - val_accuracy: 0.2670 - val_loss: 25.2375
Epoch 5/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 29s 317ms/step - accuracy: 0.9917 - loss: 0.0438 - val_accuracy: 0.2725 - val_loss: 21.6847
Epoch 6/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 320ms/step - accuracy: 0.9993 - loss: 0.0043 - val_accuracy: 0.3706 - val_loss: 16.6863
Epoch 7/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 30s 327ms/step - accuracy: 0.9987 - loss: 0.0065 - val_accuracy: 0.4632 - val_loss: 12.5658
Epoch 8/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 29s 318ms/step - accuracy: 0.9985 - loss: 0.0059 -

In [ ]:
test_loss, test_acc = model.evaluate(test_data)

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - accuracy: 0.7950 - loss: 3.3691


In [ ]:
y_true = np.concatenate([y for x, y in test_data], axis=0)
y_pred = np.argmax(model.predict(test_data), axis=1)

23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 232ms/step


In [ ]:
print(classification_report(y_true, y_pred, target_names=ds_info.features['label'].names))

              precision    recall  f1-score   support

   dandelion       0.82      0.82      0.82       159
       daisy       0.76      0.78      0.77       120
      tulips       0.81      0.78      0.80       186
  sunflowers       0.86      0.77      0.81       138
       roses       0.73      0.83      0.78       131

    accuracy                           0.80       734
   macro avg       0.80      0.80      0.80       734
weighted avg       0.80      0.80      0.80       734

